In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable
import matplotlib.pyplot as plt


ImportError: dlopen(/Users/kimjihun/opt/anaconda3/lib/python3.8/site-packages/torch/_C.cpython-38-darwin.so, 2): Library not loaded: @rpath/libtorch_cpu.dylib
  Referenced from: /Users/kimjihun/opt/anaconda3/lib/python3.8/site-packages/torch/lib/libtorch_python.dylib
  Reason: image not found

In [2]:
mnist_train = dset.MNIST("", train=True, transform=transforms.ToTensor(), target_transform=None, download=True)
mnist_test = dset.MNIST("", train=False, transform=transforms.ToTensor(), target_transform=None, download=True)

NameError: name 'dset' is not defined

In [3]:
print("mnist train 길이 : ", len(mnist_train))
print("mnist test 길이 : ", len(mnist_test))

image, label = mnist_train.__getitem__(0)
print("Image data 형태 : ", image.size())
print("Label : ", label)

img - image.numpy()
plt.title("Label : %d"%label)
plt.imshow(img[0], cmap='gray')
plt.show()

NameError: name 'mnist_train' is not defined

In [4]:
print(mnist_train[0][1])
print(mnist_train[0][0].size())

for i in range(3):
    img=mnist_train[i][0].numpy()
    print(mnist_train[i][1])
    plt.imshow(img[0], cmap='gray')
    plt.show()

NameError: name 'mnist_train' is not defined

In [5]:
image, label = mnist_train[0]
image = image.view(-1, image.size()[0], image.size()[1], image.size()[2])
print(image.size())
print(label)

conv_layer = nn.Conv2d(in_channels=1, out_channels=3, kernel_size=3, padding=1)
output = conv_layer(Variable(image))
print(output.size())

for i in range(3):
    plt.imshow(output[0,i,:,:].data.numpy(), cmap='gray')
    plt.show()

NameError: name 'mnist_train' is not defined

In [6]:
import numpy as np
import torch.optim as optim

batch_size = 16
learning_rate=0.0002
num_epoch=10

train_loader = torch.utils.data.DataLoader(list(mnist_train)[:batch_size*100], batch_size=batch_size,
                                          shuffle=True, num_workers=2, drop_last=True)
test_loader = torch.utils.data.DataLoader((mnist_test), batch_size=batch_size,
                                          shuffle=False, num_workers=2, drop_last=True)

ImportError: dlopen(/Users/kimjihun/opt/anaconda3/lib/python3.8/site-packages/torch/_C.cpython-38-darwin.so, 2): Library not loaded: @rpath/libtorch_cpu.dylib
  Referenced from: /Users/kimjihun/opt/anaconda3/lib/python3.8/site-packages/torch/lib/libtorch_python.dylib
  Reason: image not found

In [7]:
class CNN(nn.module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer = nn.Sequentail(
                        nn.Conv2d(1,16,5, padding=2),
                        nn.ReLU(),
            
                        nn.Conv2d(16,32,5, padding=2),
                        nn.ReLU(),
                        nn.MaxPool2d(2,2),
            
            
                        nn.Conv2d(32,64,5, padding=2),
                        nn.ReLU(),
                        nn.MaxPool2d(2,2),
                    )
        self.fc_layer = nn.Sequential(
            nn.Linear(64*7*7, 100),
            nn.ReLU(),
            nn.Linear(100,10)
        )
    def forward(self, x):
        out = self.layer(x)
        out = out.view(batch_size, -1)
        out = self.fc_layer(out)
        return out
    
model = CNN().cuda()

NameError: name 'nn' is not defined

In [8]:
for parameter in model.parameters():
    print(parameter.shape)

NameError: name 'model' is not defined

In [9]:
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

NameError: name 'nn' is not defined

In [10]:
for i in range(num_epoch):
    for j, [image, label] in enumerate(train_loader):
        x = Variable(image).cuda()
        y_ = Variable(label).cuda()
        
        optimizer.zero_grad()
        output = model.forward(x)
        loss = loss_func(output, y_)
        loss.backward()
        optimizer.step()
        
        if j%50 == 0:
            print(loss, j, i)

NameError: name 'num_epoch' is not defined

In [11]:
torch.save(model, './nets/cnn_mnist_%d.pkl'%(num_eopch))

NameError: name 'torch' is not defined

In [12]:
try:
    model = torch.load('./nets/cnn_mnist_10.pkl')
    print("model restored")
except Exception as e:
    print(e)

name 'torch' is not defined


In [ ]:
def ComputeAccr(dloader, imodel):
    correct = 0
    total = 0
    
    for j, [imgs, labels] in enumerate(dloader):
        img = Variable(i)